# Experiment

In [3]:
print("Start")

Start


# AlexNet

In [4]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Function to copy images to respective directories
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (227, 227, 3)  # Input shape for AlexNet
batch_size = 32
epochs = 20

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Build the AlexNet model
model_alexnet = Sequential()

# Layer 1: Convolutional Layer
model_alexnet.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
model_alexnet.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Layer 2: Convolutional Layer
model_alexnet.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
model_alexnet.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Layer 3: Convolutional Layer
model_alexnet.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 4: Convolutional Layer
model_alexnet.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 5: Convolutional Layer
model_alexnet.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model_alexnet.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Flatten the output
model_alexnet.add(Flatten())

# Fully Connected Layers
model_alexnet.add(Dense(4096, activation='relu'))

model_alexnet.add(Dense(4096, activation='relu'))

model_alexnet.add(Dense(3, activation='softmax'))  # Adjust for the number of classes

# Compile the model
model_alexnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model with class weights
history_alexnet = model_alexnet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)

# Evaluate the model on the test set
loss, accuracy = model_alexnet.evaluate(test_generator)
print('Loss:', loss)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 20s 394ms/step - accuracy: 0.4393 - loss: 1.0485 - val_accuracy: 0.4494 - val_loss: 0.8981
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 360ms/step - accuracy: 0.4991 - loss: 0.9205 - val_accuracy: 0.4051 - val_loss: 0.9306
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 363ms/step - accuracy: 0.4763 - loss: 0.8022 - val_accuracy: 0.4810 - val_loss: 0.7535
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 364ms/step - accuracy: 0.5052 - loss: 0.7649 - val_accuracy: 0.5759 - val_loss: 0.7625
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 365ms/step - accuracy: 0.5327 - loss: 0.7368 - val_accuracy: 0.5380 - val_loss: 0.7813
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 363ms/step - accuracy: 0.5874 - loss: 0.6928 - val_accuracy: 0.5506 - val_loss: 0.7337
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 366ms/step - accuracy: 0.6323 - loss: 0.6590 - val_accuracy: 0.7089 - val_loss: 0.6842
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 372ms/step - accuracy: 0.5816 - loss: 0.6889 - val_accuracy: 0.639

# EfficientNetB0

In [5]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0  # Import EfficientNet
from sklearn.utils.class_weight import compute_class_weight

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Function to copy images to respective directories
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)  # EfficientNet typically needs larger input sizes, adjust if needed
batch_size = 32
epochs = 20

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Load pre-trained EfficientNet model + higher-level layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)  # Adjust accordingly

# Freeze the base model
base_model.trainable = False

# Add Global Average Pooling and output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(3, activation='softmax')(x)  # Adjust the number of classes if needed

# Create the complete model
model_efficientnet = Model(inputs=base_model.input, outputs=output)

# Compile the model
model_efficientnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model with class weights
history_efficientnet = model_efficientnet.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)

# Evaluate the model on the test set
loss, accuracy = model_efficientnet.evaluate(test_generator)
print('Loss:', loss)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 54s 840ms/step - accuracy: 0.1752 - loss: 1.0919 - val_accuracy: 0.3418 - val_loss: 1.1070
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.3386 - loss: 1.0773 - val_accuracy: 0.2975 - val_loss: 1.1022
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.2945 - loss: 1.1189 - val_accuracy: 0.2658 - val_loss: 1.1023
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 201ms/step - accuracy: 0.3037 - loss: 1.1130 - val_accuracy: 0.5633 - val_loss: 1.0958
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 201ms/step - accuracy: 0.3224 - loss: 1.1210 - val_accuracy: 0.1772 - val_loss: 1.1002
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 208ms/step - accuracy: 0.2930 - loss: 1.1013 - val_accuracy: 0.2658 - val_loss: 1.0949
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 203ms/step - accuracy: 0.3317 - loss: 1.1092 - val_accuracy: 0.2658 - val_loss: 1.0982
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.3936 - loss: 1.0728 - val_accuracy: 0.3797 - val

# CNN Layers

In [6]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

# Define paths
dataset_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'
base_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Ensure clean split directories
for dir_path in [train_dir, val_dir, test_dir]:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)

# Split dataset into 70% train, 20% test, 10% validation
all_images = []
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    all_images.extend([(img, class_name) for img in images])

test_split = 0.2
val_split = 0.1
train_images, temp_images = train_test_split(
    all_images,
    test_size=(test_split + val_split),
    stratify=[label for _, label in all_images],
    random_state=42
)
val_images, test_images = train_test_split(
    temp_images,
    test_size=(test_split / (test_split + val_split)),
    stratify=[label for _, label in temp_images],
    random_state=42
)

# Function to copy images to respective directories
def copy_images(images, target_dir):
    for img_path, label in images:
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(img_path, label_dir)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

# Parameters
input_shape = (128, 128, 3)  # Input shape for the images
batch_size = 32
epochs = 20

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Print the number of samples in each split
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

# Compute class weights
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Print the calculated class weights
print("Class Weights:", class_weights)

# Build a simple CNN model
model_cnn = Sequential()

# Convolutional layers
model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model_cnn.add(Flatten())

# Fully connected layers
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))  # Dropout for regularization
model_cnn.add(Dense(3, activation='softmax'))  # Adjust for the number of classes

# Compile the model
model_cnn.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model with class weights
history_cnn = model_cnn.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights
)

# Evaluate the model on the test set
loss, accuracy = model_cnn.evaluate(test_generator)
print('Loss:', loss)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Found 1104 images belonging to 3 classes.
Found 158 images belonging to 3 classes.
Found 316 images belonging to 3 classes.
Training samples: 1104
Validation samples: 158
Test samples: 316
Class Weights: {0: 0.5906902086677368, 1: 1.2474576271186442, 2: 1.978494623655914}


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 302ms/step - accuracy: 0.3373 - loss: 1.0733 - val_accuracy: 0.3861 - val_loss: 1.0230
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - accuracy: 0.4358 - loss: 1.0259 - val_accuracy: 0.6519 - val_loss: 0.9208
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.5382 - loss: 0.9953 - val_accuracy: 0.5506 - val_loss: 0.8821
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.5532 - loss: 0.9358 - val_accuracy: 0.5316 - val_loss: 0.8697
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.5667 - loss: 0.8545 - val_accuracy: 0.6329 - val_loss: 0.8007
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.5430 - loss: 0.8549 - val_accuracy: 0.6013 - val_loss: 0.8095
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.5753 - loss: 0.8029 - val_accuracy: 0.5696 - val_loss: 0.8480
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 9s 204ms/step - accuracy: 0.6092 - loss: 0.7801 - val_accuracy: 0.6266 - val